In [11]:
import os
import pandas as pd
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
from tagger.core.mongo.models.order import Order
from pytz import timezone
from datetime import datetime
from tagger.models.inventory import Inventory, InventoryStatus


TZ = timezone("Asia/Seoul")
# TARGETS = [
#     "61812c7d26685ff051754cee",
# ]

PENDING_FOR_BUNDLING = [
    "6187956f26685ff051755922"
]
to_be_shipped = Inventory.objects.filter(
    status=InventoryStatus.SHIPPING_PENDING
).exclude(
    out_order_id__in=PENDING_FOR_BUNDLING
).all()
TARGETS = [x.out_order_id for x in to_be_shipped]


CJ_COLUMNS = [
    "주문번호", "주문일", "수령자명", "수령자 우편번호", "수령자 연락처", "수령자주소", "상품명", "옵션명", "수량", "배송메모", "박스타입"
]



def get_cj_row(order):
    payment = order.payment
    return [
        order.code.replace("ORD-", ""),
        order.created.astimezone(TZ).date().isoformat().replace("-", "/"),
        payment.buyername,
        payment.buyerpostcode,
        f"{payment.buyermobile[:3]}-{payment.buyermobile[3:7]}-{payment.buyermobile[7:11]}",
        payment.buyeraddress,
        payment.name,
        ",".join([f"{item.size}/{item.quantity}EA" for item in order.orders]),
        1,
        order.memo,
        "소"
    ]

def get_cu_row(order):
    payment = order.get_payment()
    return [
        payment.buyername,
        payment.buyerpostcode,
        payment.buyeraddress,
        payment.buyeraddress,
        f"{payment.buyermobile[:3]}-{payment.buyermobile[3:7]}-{payment.buyermobile[7:11]}",
        "",
        order.memo,
        "선불"
    ]

def copy_cu():
    orders = Order.objects(id__in=TARGETS)
    df = pd.DataFrame([get_cu_row(order) for order in orders])
    df[1] = df[1].apply(lambda x: x.zfill(5))
    df.to_clipboard()

def make_cj():
    orders = Order.objects(id__in=TARGETS)
    raw_items = [get_cj_row(order) for order in orders]
    
    # 합배처리
    item_map = {}
    for item in raw_items:
        key = (item[2], item[3], item[4])  # name, post, mobile
        if key not in item_map:
            item_map[key] = item
        else:
            new_item = [*item_map[key]]
            new_item[0] = new_item[0] + item[0]  # order code
            new_item[6] = new_item[6] + ", " + item[6]  # payment name
            new_item[7] = new_item[7] + ", " + item[7]  # sizes
            item_map[key] = new_item
            
        
    df = pd.DataFrame(item_map.values(), columns=CJ_COLUMNS)
    df["수령자 우편번호"] = df["수령자 우편번호"].apply(lambda x: x.zfill(5))
    with open(f"../Downloads/cj_upload_{datetime.now().date().isoformat()}.xlsx", "wb") as file:
        df.to_excel(file, index=None)


In [13]:
 for o in Order.objects(id__in=TARGETS).all():
    print(o.payment.buyername, f"https://office.alloff.co/orders/{o.id}")


주보배 https://office.alloff.co/orders/617a795b17fc2e5a1f9eab42
윤인주 https://office.alloff.co/orders/617fcaf517fc2e5a1f9eb2e6
하준희 https://office.alloff.co/orders/6180d7e817fc2e5a1f9eb4ea
정민지 https://office.alloff.co/orders/6183a76a26685ff051755184
윤경 https://office.alloff.co/orders/6183a95226685ff05175518d
신지혜 https://office.alloff.co/orders/6183b82426685ff0517551bc
김미진 https://office.alloff.co/orders/6184d8b326685ff05175540a
김주연 https://office.alloff.co/orders/6184ef7e26685ff05175542f
홍인경  https://office.alloff.co/orders/6185018c26685ff051755440
김미진 https://office.alloff.co/orders/6185165726685ff0517554c2
민은정 https://office.alloff.co/orders/618516ef26685ff0517554c5
민은정 https://office.alloff.co/orders/61851cfd26685ff0517554eb
채은희 https://office.alloff.co/orders/6185323426685ff051755544
이정원 https://office.alloff.co/orders/61853ac026685ff051755565
이정원 https://office.alloff.co/orders/61853b3026685ff051755569
정나영 https://office.alloff.co/orders/6185ace326685ff051755600
김민선 https://office.allof

FileNotFoundError: [Errno 2] No such file or directory: '~/cj_upload.xls'

In [8]:
make_cj()

'43D3W'